In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split

import gc
import os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)

In [2]:
PATH_TO_DATA = ('D:/Py/DataFrames/Home_Credit_Default_Risk(KAGGLE)/')

# LGB KFOLD

In [3]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()

In [4]:
def kfold_lightgbm(df, num_folds, stratified=False, submission_file_name='random_sumbit.csv'):
    metric_scores = {}
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    test_df.drop('TARGET', axis=1, inplace=True)
    del df
    gc.collect()
    
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=44)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=44)
        
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
        
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=28,
            colsample_bytree=0.5,
            subsample=0.8715623,
            max_depth=-1,
            reg_alpha=0,
            reg_lambda=100,
            min_split_gain=0.5,
            min_child_samples=70,
            silent=-1,
            verbose=-1)
        
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
                eval_metric='auc', verbose=100, early_stopping_rounds=100)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        metric_scores[n_fold + 1] = roc_auc_score(valid_y, oof_preds[valid_idx])
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
        
    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('CV SCORES: ', metric_scores)
    # Write submission file and plot feature importance
    test_df['TARGET'] = sub_preds
    test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index=False)
    
    return feature_importance_df

# Pooling DataFrames

In [6]:
%%time
df = pd.read_csv(os.path.join(PATH_TO_DATA, 'df_all_files_with_fs.csv'))
app = pd.read_csv(os.path.join(PATH_TO_DATA, 'application_diff_agg_features.csv'))
app.drop(['TARGET'], axis=1, inplace=True)
df = df.merge(app, on='SK_ID_CURR', how='left')
del app
gc.collect()

bureau = pd.read_csv(os.path.join(PATH_TO_DATA, 'bureau_agg_features.csv'))
df = df.merge(bureau, on='SK_ID_CURR', how='left')
del bureau
gc.collect()

prev = pd.read_csv(os.path.join(PATH_TO_DATA, 'prev_application_agg_features.csv'))
df = df.merge(prev, on='SK_ID_CURR', how='left')
del prev
gc.collect()

pos = pd.read_csv(os.path.join(PATH_TO_DATA, 'pos_cash_agg_features.csv'))
df = df.merge(pos, on='SK_ID_CURR', how='left')
del pos
gc.collect()

ins = pd.read_csv(os.path.join(PATH_TO_DATA, 'installments_agg_features.csv'))
df = df.merge(ins, on='SK_ID_CURR', how='left')
del ins
gc.collect()

cc = pd.read_csv(os.path.join(PATH_TO_DATA, 'credit_card_agg_features.csv'))
df = df.merge(cc, on='SK_ID_CURR', how='left')
del cc
gc.collect()

print('Full DF shape: ', df.shape)

Full DF shape:  (356251, 944)
Wall time: 2min 32s


---

In [7]:
prev = pd.read_csv(os.path.join(PATH_TO_DATA, 'previous_application.csv'))

In [8]:
tmp = prev.groupby('SK_ID_CURR').agg({'SK_ID_PREV': 'nunique',
                                      'NAME_CONTRACT_TYPE': 'nunique'}) \
            .reset_index().rename(index=str, columns={'SK_ID_PREV': 'count_of_prev_credits',
                                                      'NAME_CONTRACT_TYPE': 'count_of_types_prev_credits'})
df = df.merge(tmp, on='SK_ID_CURR', how='left')

prev['NAME_YIELD_GROUP'].replace({'XNA': 0, 'low_normal': 0.2, 'low_action': 0.3, 'middle': 0.5, 'high': 0.7}, inplace=True)
tmp = prev.groupby('SK_ID_CURR')['NAME_YIELD_GROUP'].last().reset_index() \
            .rename(index=str, columns={'NAME_YIELD_GROUP': 'last_yield_group_prev_credit'})
df = df.merge(tmp, on='SK_ID_CURR', how='left')

del prev, tmp
gc.collect()

35

In [9]:
ins = pd.read_csv(os.path.join(PATH_TO_DATA, 'installments_payments.csv'))

ins['inst_paid_late_in_days'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
ins['inst_paid_over_amount'] = ins['AMT_PAYMENT'] - ins['AMT_INSTALMENT']

In [10]:
for i in range(3):
    tmp = ins.groupby('SK_ID_PREV')['inst_paid_late_in_days'].nth(i).reset_index() \
                .rename(index=str, columns={'inst_paid_late_in_days':'inst_{}_late_in_days'.format(i+1)})
    ins = ins.merge(tmp, on='SK_ID_PREV', how='left')
    
    tmp = ins.groupby('SK_ID_PREV')['inst_paid_over_amount'].nth(i).reset_index() \
                .rename(index=str, columns={'inst_paid_over_amount':'inst_{}_paid_over_amount'.format(i+1)})
    ins = ins.merge(tmp, on='SK_ID_PREV', how='left')

In [11]:
for i in range(3):
    for agg in ['min', 'max', 'mean', 'median']:
        tmp = ins.groupby('SK_ID_CURR').agg({
            'inst_{}_late_in_days'.format(i+1): agg,
            'inst_{}_paid_over_amount'.format(i+1): agg}).reset_index()
        tmp = tmp.rename(index=str, columns={
            'inst_{}_late_in_days'.format(i+1): 'inst_{}_late_in_days_{}'.format(i+1, agg),
            'inst_{}_paid_over_amount'.format(i+1): 'inst_{}_paid_over_amount_{}'.format(i+1, agg)})
        df = df.merge(tmp, on='SK_ID_CURR', how='left')

In [12]:
del ins, tmp
gc.collect()

112

In [13]:
# binary
df['existence_credit_history'] = 1
df.loc[df['count_of_prev_credits'].isnull(), 'existence_credit_history'] = 0
df['ext_source_1_isnull'] = 0
df.loc[df['EXT_SOURCE_1'].isnull(), 'ext_source_1_isnull'] = 1
df['ext_source_2_isnull'] = 0
df.loc[df['EXT_SOURCE_2'].isnull(), 'ext_source_2_isnull'] = 1
df['ext_source_3_isnull'] = 0
df.loc[df['EXT_SOURCE_3'].isnull(), 'ext_source_3_isnull'] = 1

df['income/12_annuity_gap'] = df['AMT_INCOME_TOTAL'] / 12. - df['AMT_ANNUITY']

In [14]:
df.columns.tolist()

['ACTIVE_AMT_ANNUITY_sum',
 'ACTIVE_AMT_CREDIT_MAX_OVERDUE_mean',
 'ACTIVE_AMT_CREDIT_SUM_DEBT_mean',
 'ACTIVE_AMT_CREDIT_SUM_LIMIT_mean',
 'ACTIVE_AMT_CREDIT_SUM_OVERDUE_mean',
 'ACTIVE_AMT_CREDIT_SUM_mean',
 'ACTIVE_AMT_CREDIT_SUM_sum',
 'ACTIVE_CNT_CREDIT_PROLONG_sum',
 'ACTIVE_CREDIT_ACTIVE_BINARY_mean',
 'ACTIVE_CREDIT_DAY_OVERDUE_mean',
 'ACTIVE_CREDIT_ENDDATE_BINARY_mean',
 'ACTIVE_DAYS_CREDIT_ENDDATE_mean',
 'ACTIVE_DAYS_CREDIT_UPDATE_mean',
 'ACTIVE_DAYS_CREDIT_count',
 'ACTIVE_DAYS_CREDIT_mean',
 'ACTIVE_DAYS_CREDIT_var',
 'ACTIVE_MONTHS_BALANCE_MAX_max',
 'ACTIVE_MONTHS_BALANCE_MIN_min',
 'ACTIVE_MONTHS_BALANCE_SIZE_mean',
 'ACTIVE_MONTHS_BALANCE_SIZE_sum',
 'AMT_ANNUITY',
 'AMT_CREDIT',
 'AMT_INCOME_TOTAL',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'ANNUITY_GOODS_PRICE_PERC',
 'ANNUITY_INCOME_PERC',
 'APARTMENTS_AVG',
 'APPROVED_AMT_A

In [15]:
df.shape

(356251, 976)

# Feature Selection

1) Remove missing values

In [16]:
train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]
test.drop('TARGET', axis=1, inplace=True)
del df
gc.collect()

7

In [17]:
train_missing = (train.isnull().sum() / len(train)).sort_values(ascending = False)
train_missing.head()

prev_app_days_first_drawing_about_last_1_credits_mean    0.967952
prev_app_days_first_drawing_about_last_3_credits_mean    0.903404
prev_app_days_first_drawing_about_last_5_credits_mean    0.868757
ccb_active_contracts_perc                                0.855278
ccb_completed_contracts_perc                             0.855278
dtype: float64

In [18]:
test_missing = (test.isnull().sum() / len(test)).sort_values(ascending = False)
test_missing.head()

prev_app_days_first_drawing_about_last_1_credits_mean    0.971032
prev_app_days_first_drawing_about_last_3_credits_mean    0.902511
prev_app_days_first_drawing_about_last_5_credits_mean    0.864086
ccb_completed_contracts_perc                             0.792631
ccb_active_contracts_perc                                0.792631
dtype: float64

In [19]:
train_missing = train_missing.index[train_missing > 0.80]
test_missing = test_missing.index[test_missing > 0.80]

all_missing = list(set(set(train_missing) | set(test_missing)))
print('There are %d columns with more than 80%% missing values' % len(all_missing))

There are 27 columns with more than 80% missing values


In [20]:
train.drop(all_missing, axis=1, inplace=True)
test.drop(all_missing, axis=1, inplace=True)
print(train.shape, test.shape)

(307507, 949) (48744, 948)


2) Remove Collinear Variables

In [21]:
# i am dealing it with such a limited section of the data (~33%) because of waiting
mini_train = train.sample(100000)

In [22]:
%%time
# Absolute value correlation matrix
corr_matrix = mini_train.corr().abs()

Wall time: 11min 40s


In [23]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head(3)

,ACTIVE_AMT_ANNUITY_sum,ACTIVE_AMT_CREDIT_MAX_OVERDUE_mean,ACTIVE_AMT_CREDIT_SUM_DEBT_mean,ACTIVE_AMT_CREDIT_SUM_LIMIT_mean,ACTIVE_AMT_CREDIT_SUM_OVERDUE_mean,ACTIVE_AMT_CREDIT_SUM_mean,ACTIVE_AMT_CREDIT_SUM_sum,ACTIVE_CNT_CREDIT_PROLONG_sum,ACTIVE_CREDIT_ACTIVE_BINARY_mean,ACTIVE_CREDIT_DAY_OVERDUE_mean,ACTIVE_CREDIT_ENDDATE_BINARY_mean,ACTIVE_DAYS_CREDIT_ENDDATE_mean,ACTIVE_DAYS_CREDIT_UPDATE_mean,ACTIVE_DAYS_CREDIT_count,ACTIVE_DAYS_CREDIT_mean,ACTIVE_DAYS_CREDIT_var,ACTIVE_MONTHS_BALANCE_MAX_max,ACTIVE_MONTHS_BALANCE_MIN_min,ACTIVE_MONTHS_BALANCE_SIZE_mean,ACTIVE_MONTHS_BALANCE_SIZE_sum,AMT_ANNUITY,AMT_CREDIT,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,ANNUITY_GOODS_PRICE_PERC,ANNUITY_INCOME_PERC,APARTMENTS_AVG,APPROVED_AMT_APPLICATION_MAX,APPROVED_AMT_APPLICATION_MEAN,APPROVED_CNT_PAYMENT_MEAN,APPROVED_CNT_PAYMENT_SUM,APPROVED_DAYS_DECISION_MAX,BASEMENTAREA_AVG,BUREAU_AMT_ANNUITY_MAX,BUREAU_AMT_ANNUITY_MEAN,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN,BUREAU_AMT_CREDIT_SUM_LIMIT_SUM,BUREAU_AMT_CREDIT_SUM_MEAN,BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_OVERDUE_SUM,BUREAU_AMT_CREDIT_SUM_SUM,BUREAU_CNT_CREDIT_PROLONG_SUM,BUREAU_CREDIT_ACTIVE_BINARY_MEAN,BUREAU_CREDIT_ACTIVE_Bad debt_MEAN,BUREAU_CREDIT_ACTIVE_Sold_MEAN,BUREAU_CREDIT_ACTIVE_nan_MEAN,BUREAU_CREDIT_CURRENCY_currency 1_MEAN,BUREAU_CREDIT_CURRENCY_currency 3_MEAN,BUREAU_CREDIT_CURRENCY_currency 4_MEAN,BUREAU_CREDIT_CURRENCY_nan_MEAN,BUREAU_CREDIT_DAY_OVERDUE_MEAN,BUREAU_CREDIT_ENDDATE_BINARY_MEAN,BUREAU_CREDIT_TYPE_Another type of loan_MEAN,BUREAU_CREDIT_TYPE_Car loan_MEAN,BUREAU_CREDIT_TYPE_Cash loan (non-earmarked)_MEAN,BUREAU_CREDIT_TYPE_Consumer credit_MEAN,BUREAU_CREDIT_TYPE_Credit card_MEAN,BUREAU_CREDIT_TYPE_Interbank credit_MEAN,BUREAU_CREDIT_TYPE_Loan for business development_MEAN,BUREAU_CREDIT_TYPE_Loan for purchase of shares (margin lending)_MEAN,BUREAU_CREDIT_TYPE_Loan for the purchase of equipment_MEAN,BUREAU_CREDIT_TYPE_Loan for working capital replenishment_MEAN,BUREAU_CREDIT_TYPE_Microloan_MEAN,BUREAU_CREDIT_TYPE_Mobile operator loan_MEAN,BUREAU_CREDIT_TYPE_Mortgage_MEAN,BUREAU_CREDIT_TYPE_Real estate loan_MEAN,BUREAU_CREDIT_TYPE_Unknown type of loan_MEAN,BUREAU_CREDIT_TYPE_nan_MEAN,BUREAU_DAYS_CREDIT_COUNT,BUREAU_DAYS_CREDIT_ENDDATE_MEAN,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_UPDATE_MEAN,BUREAU_DAYS_CREDIT_VAR,BUREAU_MONTHS_BALANCE_MAX_MAX,BUREAU_MONTHS_BALANCE_MIN_MIN,BUREAU_MONTHS_BALANCE_SIZE_MEAN,BUREAU_MONTHS_BALANCE_SIZE_SUM,BUREAU_STATUS_0_MEAN_MEAN,BUREAU_STATUS_1_MEAN_MEAN,BUREAU_STATUS_2_MEAN_MEAN,BUREAU_STATUS_3_MEAN_MEAN,BUREAU_STATUS_4_MEAN_MEAN,BUREAU_STATUS_5_MEAN_MEAN,BUREAU_STATUS_C_MEAN_MEAN,BUREAU_STATUS_X_MEAN_MEAN,BUREAU_STATUS_nan_MEAN_MEAN,CC_AMT_BALANCE_max,CC_AMT_BALANCE_mean,CC_AMT_BALANCE_sum,CC_AMT_BALANCE_var,CC_AMT_CREDIT_LIMIT_ACTUAL_max,CC_AMT_CREDIT_LIMIT_ACTUAL_sum,CC_AMT_CREDIT_LIMIT_ACTUAL_var,CC_AMT_DRAWINGS_ATM_CURRENT_max,CC_AMT_DRAWINGS_ATM_CURRENT_mean,CC_AMT_DRAWINGS_ATM_CURRENT_sum,CC_AMT_DRAWINGS_ATM_CURRENT_var,CC_AMT_DRAWINGS_CURRENT_max,CC_AMT_DRAWINGS_CURRENT_mean,CC_AMT_DRAWINGS_CURRENT_sum,CC_AMT_DRAWINGS_CURRENT_var,CC_AMT_DRAWINGS_OTHER_CURRENT_sum,CC_AMT_DRAWINGS_POS_CURRENT_sum,CC_AMT_PAYMENT_TOTAL_CURRENT_max,CC_AMT_PAYMENT_TOTAL_CURRENT_mean,CC_AMT_PAYMENT_TOTAL_CURRENT_var,CC_CNT_DRAWINGS_ATM_CURRENT_sum,CC_CNT_DRAWINGS_CURRENT_max,CC_CNT_DRAWINGS_CURRENT_mean,CC_CNT_DRAWINGS_CURRENT_sum,CC_CNT_DRAWINGS_CURRENT_var,CC_CNT_DRAWINGS_OTHER_CURRENT_sum,CC_CNT_INSTALMENT_MATURE_CUM_max,CC_MONTHS_BALANCE_max,CC_MONTHS_BALANCE_mean,CC_NAME_CONTRACT_STATUS_Active_max,CC_NAME_CONTRACT_STATUS_Active_mean,CC_NAME_CONTRACT_STATUS_Active_var,CC_NAME_CONTRACT_STATUS_Approved_max,CC_NAME_CONTRACT_STATUS_Completed_max,CC_NAME_CONTRACT_STATUS_Completed_sum,CC_NAME_CONTRACT_STA

In [24]:
threshold = 0.90

to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print('There are %d columns to remove.' % (len(to_drop)))

There are 221 columns to remove.


In [25]:
to_drop

['BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN',
 'CC_NAME_CONTRACT_STATUS_Active_var',
 'INSTAL_AMT_INSTALMENT_STD',
 'CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_REGION_POPULATION_RELATIVE_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_external_sources_mean_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_ANNUITY_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT',
 'CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_max_OWN_CAR_AGE_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_mean_AMT_INCOME_TOTAL_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_mean_external_sources_mean_abs_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_mean_external_sources_mean_diff',
 'CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY',
 'CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY_diff',
 'CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN_diff',
 'CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_DAYS_ID_PUBLISH',
 'CODE_GEN

In [26]:
train.drop(to_drop, axis=1, inplace=True)
test.drop(to_drop, axis=1, inplace=True)
del mini_train
gc.collect()
print(train.shape, test.shape)

(307507, 728) (48744, 727)


3) Save changes

In [27]:
test['TARGET'] = np.nan
df = train.append(test).reset_index(drop=True)
del test
gc.collect()

df.to_csv(os.path.join(PATH_TO_DATA, 'df_with_fs.csv'), index=False)

---

In [5]:
df = pd.read_csv(os.path.join(PATH_TO_DATA, 'df_with_fs.csv'))
print(df.shape)

(356251, 728)


In [6]:
%%time
feat_importance = kfold_lightgbm(df, num_folds=5, stratified=True, submission_file_name='best_solo_model.csv')

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.748741	valid_1's auc: 0.744605
[200]	training's auc: 0.773198	valid_1's auc: 0.761919
[300]	training's auc: 0.790231	valid_1's auc: 0.773215
[400]	training's auc: 0.801214	valid_1's auc: 0.779727
[500]	training's auc: 0.809467	valid_1's auc: 0.783527
[600]	training's auc: 0.816481	valid_1's auc: 0.786056
[700]	training's auc: 0.822656	valid_1's auc: 0.787907
[800]	training's auc: 0.828124	valid_1's auc: 0.789188
[900]	training's auc: 0.833141	valid_1's auc: 0.790029
[1000]	training's auc: 0.837769	valid_1's auc: 0.790583
[1100]	training's auc: 0.842116	valid_1's auc: 0.79107
[1200]	training's auc: 0.846232	valid_1's auc: 0.79155
[1300]	training's auc: 0.850188	valid_1's auc: 0.791968
[1400]	training's auc: 0.85377	valid_1's auc: 0.792227
[1500]	training's auc: 0.857329	valid_1's auc: 0.792507
[1600]	training's auc: 0.860752	valid_1's auc: 0.792758
[1700]	training's auc: 0.864086	valid_1's auc: 0.7928